In [2]:
%pip3 install vaderSentiment

     |████████████████████████████████| 125 kB 23.9 MB/s 


In [3]:
%pip3 install gensim --upgrade
%pip3 install tf-estimator-nightly==2.8.0.dev2021122109
%pip3 install keras==2.8.0 --upgrade
%pip3 install pandas --upgrade

     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 462 kB 11.1 MB/s 


In [15]:
import socket
import sys
import requests
import requests_oauthlib
import json
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import enum
class Method(enum.Enum):
   Vader = 1
   LSTM = 2
# Replace the values below with yours
ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAEt%2FcgEAAAAA%2FgThpZgB8X3Ds%2BJ20r4zjpeBpXI%3DIzrgb0nXRJf5h6Qbneg0z2rS7KLgU6NWQGgJiaqvkuf1zz4x2j'

In [16]:
headers = {"Authorization": "Bearer {}".format(BEARER_TOKEN), "content-type": "application/json"}

In [17]:
def get_tweets():
    url = 'https://api.twitter.com/2/tweets/1228393702244134912'
    query_data = [('tweet.fields', 'created_at,attachments'), ('expansions', 'author_id')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, headers=headers, stream=True)
    print(query_url, response)
    
    return response

In [18]:
def get_tweets_by_user(user):
    url = 'https://api.twitter.com/2/tweets/search/recent'
    query_data = [('query', 'from:'+user), ('max_results', '10')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, headers=headers, stream=True)
    #print(query_url, response)
    
    return response

In [19]:
def get_tweet_comments(id='1526022409383460866'):
    url = 'https://api.twitter.com/2/tweets/search/recent'
    #url = 'https://api.twitter.com/2/tweets/search/all'
    query_data = [('query', 'conversation_id:'+id), ('max_results', '10'), ('tweet.fields', 'conversation_id,in_reply_to_user_id,created_at')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, headers=headers, stream=True)
    #print(query_url, response)
    
    return response

In [20]:
analyzer = SentimentIntensityAnalyzer()

def process_tweets_comments_vader(http_resp):
    data = http_resp.json()
    negative = 0
    neutral = 0
    positive = 0
    if 'data' in data:
      for comment in data['data']:
          tweet_text = str(comment['text'].encode("utf-8"))

          # analysis sentiment score
          sentiment_score = analyzer.polarity_scores(tweet_text)["compound"]
          if sentiment_score >= 0.05:
              sentiment = "POSITIVE"
              positive += 1
          elif sentiment_score <= -0.05:
              sentiment = "NEGATIVE"
              negative += 1
          else:
              sentiment = "NEUTRAL"
              neutral += 1

          # separate sentiment label with tweet content
          mess =  sentiment + '||||' + tweet_text + '\n'
          #print(mess)
          #tcp_connection.send(bytes(mess, 'utf-8'))
    return (positive,neutral,negative)

In [28]:
from keras.preprocessing.sequence import pad_sequences

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (c:\Python310\lib\site-packages\keras\preprocessing\sequence.py)

In [21]:
# DataFrame
import pandas as pd
import tensorflow as tf
from tensorflow import keras
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
# Keras
from keras.preprocessing.sequence import pad_sequences
# Word2vec
import gensim
# Utility
import re
import logging
import time
import pickle
import string
# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# TEXT CLEANING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)
SEQUENCE_LENGTH = 300
# EXPORT FILE
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.w2v"
TOKENIZER_MODEL = "tokenizer.pkl"

nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
w2v_model = gensim.models.word2vec.Word2Vec.load("/content/drive/MyDrive/Thac_sy/HK3/HTTM/BTL/model.w2v")
tokenizer = pickle.load(open('/content/drive/MyDrive/Thac_sy/HK3/HTTM/BTL/tokenizer.pkl','rb'))
model = keras.models.load_model("/content/drive/MyDrive/Thac_sy/HK3/HTTM/BTL/model.h5")

def remove_hashtag(sentence):
  return re.sub("([@#][A-Za-z0-9_]+)","", sentence)

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    # Remove hashtag
    text = re.sub("([@#][A-Za-z0-9_]+)","", text)
    # Remove number
    text = re.sub("(\d+)", '', text)
    # Remove punctuation
    punctuation = list(string.punctuation) + ['...',  '``']
    tokens = []
    for token in text.split():
        if (token not in stop_words) and (token not in punctuation):
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

def predict(text, include_neutral=True):
    start_at = time.time()
    # Prepocess
    text = preprocess(text)
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}

def process_tweets_comments_lstm(http_resp):
    data = http_resp.json()
    negative = 0
    neutral = 0
    positive = 0
    if 'data' in data:
      for comment in data['data']:
          tweet_text = str(comment['text'].encode("utf-8"))

          # analysis sentiment score
          label = predict(tweet_text)['label']
          if label == "POSITIVE":
              positive += 1
          elif label == "NEGATIVE":
              negative += 1
          else:
              neutral += 1
          mess =  label + '||||' + tweet_text + '\n'
          #print(mess)
    return (positive,neutral,negative)

ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (c:\Python310\lib\site-packages\keras\preprocessing\sequence.py)

In [26]:
def analyze_sentimental_recent_post_of_user(username, method=Method.Vader):
    print('Analyzing for user %s ...' %(username))
    resp_tw = get_tweets_by_user(username)
    data = resp_tw.json()
    print(data)
    positive = 0
    neutral = 0
    negative = 0
    if 'data' in data:
        for tweet in data['data']:
            #print(tweet['id'])
            resp= get_tweet_comments(tweet['id'])
            #print(resp)
            if method == Method.Vader:
              positive,neutral,negative = process_tweets_comments_vader(resp)
            else:
              positive,neutral,negative = process_tweets_comments_lstm(resp)
            print('TWEET: %s \nPOS: %d, NEU: %d, NEG: %d'%(tweet['text'],positive,neutral,negative))

In [35]:
analyze_sentimental_recent_post_of_user('POTUS', Method.Vader)

Analyzing for user POTUS ...
{'data': [{'id': '1526744379708293121', 'text': 'Today marks 68 years since Brown v. Board of Education — outlawing racial segregation in schools and bringing us closer to equal protection under the law. As we celebrate the landmark decision, we must continue our work to build a more just and equitable future for all Americans.'}, {'id': '1526736453140717570', 'text': 'Jill and I were honored to welcome folks to celebrate Asian American, Native Hawaiian, and Pacific Islander Heritage Month. It was a reminder that the reason America is so strong — the reason we are who we are — is because we’re diverse with so many talents. We can’t forget that. https://t.co/j7ZDKgZUFN'}, {'id': '1526699586659749888', 'text': 'Hate and fear are being given too much oxygen by those who pretend to love America, but who don’t understand America.\n\nTo confront the ideology of hate requires caring about all people. That’s the America I know: the most diverse, multi-racial, dynam

In [36]:
analyze_sentimental_recent_post_of_user('POTUS', Method.LSTM)

Analyzing for user POTUS ...
{'data': [{'id': '1526744379708293121', 'text': 'Today marks 68 years since Brown v. Board of Education — outlawing racial segregation in schools and bringing us closer to equal protection under the law. As we celebrate the landmark decision, we must continue our work to build a more just and equitable future for all Americans.'}, {'id': '1526736453140717570', 'text': 'Jill and I were honored to welcome folks to celebrate Asian American, Native Hawaiian, and Pacific Islander Heritage Month. It was a reminder that the reason America is so strong — the reason we are who we are — is because we’re diverse with so many talents. We can’t forget that. https://t.co/j7ZDKgZUFN'}, {'id': '1526699586659749888', 'text': 'Hate and fear are being given too much oxygen by those who pretend to love America, but who don’t understand America.\n\nTo confront the ideology of hate requires caring about all people. That’s the America I know: the most diverse, multi-racial, dynam

In [37]:
predict("@POTUS Your a racist POS SWAMP RAT")

{'elapsed_time': 0.0995635986328125,
 'label': 'NEUTRAL',
 'score': 0.5466052889823914}